In [4]:
import sys
sys.path.append('..')
import csv
import numpy as np

# テキストデータからid振り分け
def text_to_id(texts):
    #words = text.split(' ')
    word_to_id = {}
    id_to_word = {}
    allwords=[]
    for words in texts:
        allwords.extend(words)
        #print("ALL",allwords)
        for word in words:
            if word not in word_to_id:
                new_id = len(word_to_id)
                word_to_id[word] = new_id
                id_to_word[new_id] = word
    corpus = np.array([word_to_id[w] for w in allwords])
    return corpus, word_to_id, id_to_word

# 単語を空白で分割 → リストに
def word_to_input(word):
    word_list = word.split(' ')
    return word_list

# リストを入力の型にはめる
def list_to_input(word_list):
    input_list = np.zeros((10), dtype=int)
    for count in range(len(word_list)):
        try:
            input_list[count] = word_to_id[word_list[count]]
        except:
            return False
    input_list = input_list[::-1]
    return input_list

def generation_text(word, model):
    word_list = word_to_input(word)
    input_list = list_to_input(word_list)
    if type(input_list) == bool:
        return '<error>：該当する単語がありませんでした。'
    else:
        input_list = np.array([input_list])
    guess_list = model.generate(input_list, 4, 49)
    guess = ''.join([id_to_word[int(c[0])] for c in guess_list])
    return guess

filename='word2text_next_txt_2.csv'
model_name='word2text_next_amazon.pkl'

# 最大入力サイズと最大出力サイズ
s1_max = 0
s2_max = 0

with open(filename, 'r', encoding='utf-8', newline=None) as tsv_file:
    end = 0
    l = []
    for row in tsv_file:
        s = row.split(',')
        s[0] = s[0].replace('\ufeff', '')
        s[1] = s[1].replace('\n', '')
        s1 = s[0].split(' ')
        s2 = s[1].split(' ')
        s1_max = max(s1_max, len(s1))
        s2.insert(0, '_')
        s2_max = max(s2_max, len(s2))
        l.append(s1)
        l.append(s2)
l[0].insert(0, '')

print('入力最大値：', s1_max)
print('出力最大値：', s2_max)

corpus_all, word_to_id, id_to_word = text_to_id(l)

print('単語の種類：', len(word_to_id))
print('総単語数：', len(corpus_all))

# x = np.zeros((72904, 10), dtype=int)
# t = np.zeros((72904, 80), dtype=int)

# for i in range(len(l)):
#     if i % 2 == 0:
#         for j in range(len(l[i])):
#             x[i//2][j] = word_to_id[l[i][j]]
#     else:
#         for j in range(len(l[i])):
#             t[i//2][j] = word_to_id[l[i][j]]

# x = x[:71407]
# t = t[:71407]
# x = x[:, ::-1]

# from common.util import eval_seq2seq
from function.attention_seq2seq import AttentionSeq2seq

# ハイパーパラメータの設定
vocab_size = len(word_to_id)
wordvec_size = 1000
hidden_size = 1000
print('実行中・・・')
model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
model.load_params(model_name)
print('実行終了')

入力最大値： 7
出力最大値： 78
単語の種類： 22397
総単語数： 1233726
実行中・・・
実行終了


In [5]:
# ここで、文章生成
word = input()
print('入力：', word)
print('出力↓')

for _ in range(10):
    print(generation_text(word, model))

入力： スキー 男 山
出力↓
人々は山でスキーに降りるためにジャンプします。
スキーの子供たちも山の湖でスキーをしています。
スキー用の山々の上をスキーする電気で女性がいます。
スキーで山でスキーをしている
山のスキーでスキーをする女性。
山のスキー場でスキーをする人。
子供たちは山の周りでスキーをします
人々は山のスキーのスキーに到達するための嵐の雲に登ります。
山のスキーのスキー・たりする女性
湖でスキーをすると人々はスキーをしてグループを山に向けました
